In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('radiomics_features_all_patients_TUMOR.csv')

In [3]:
df.head()

,PatientID,T1c_diagnostics_Versions_PyRadiomics,T1c_diagnostics_Versions_Numpy,T1c_diagnostics_Versions_SimpleITK,T1c_diagnostics_Versions_PyWavelet,T1c_diagnostics_Versions_Python,T1c_diagnostics_Configuration_Settings,T1c_diagnostics_Configuration_EnabledImageTypes,T1c_diagnostics_Image-original_Hash,T1c_diagnostics_Image-original_Dimensionality,...,MD_original_glszm_SmallAreaHighGrayLevelEmphasis,MD_original_glszm_SmallAreaLowGrayLevelEmphasis,MD_original_glszm_ZoneEntropy,MD_original_glszm_ZonePercentage,MD_original_glszm_ZoneVariance,MD_original_ngtdm_Busyness,MD_original_ngtdm_Coarseness,MD_original_ngtdm_Complexity,MD_original_ngtdm_Contrast,MD_original_ngtdm_Strength
0,UCSF-PDGM-0130_nifti,v3.1.0,1.23.5,2.5.0,1.6.0,3.9.22,"{'minimumROIDimensions': 2, 'minimumROISize': ...",{'Original': {}},81ef47d2a31deb568bf23a399324afe391e19898,3D,...,0.208002,0.208002,2.321928,0.000555,8.876046e+06,0.000000,1000000.000000,0.000000,0.000000,0.000000
1,UCSF-PDGM-0046_nifti,v3.1.0,1.23.5,2.5.0,1.6.0,3.9.22,"{'minimumROIDimensions': 2, 'minimumROISize': ...",{'Original': {}},ce2a05b786c95c45a3b6dfe67b4bd3b47d1a36a5,3D,...,2543.012756,0.001269,7.748978,0.560357,7.375384e+00,0.153713,0.001104,52593.855670,0.304477,9.420885
2,UCSF-PDGM-0132_nifti,v3.1.0,1.23.5,2.5.0,1.6.0,3.9.22,"{'minimumROIDimensions': 2, 'minimumROISize': ...",{'Original': {}},e6837ab578fa4f0f2321afe6f808cdd015b995cf,3D,...,3.659826,0.009922,2.750000,0.042553,1.075500e+03,1.752623,0.047464,3.341956,0.016476,0.493350
3,UCSF-PDGM-0107_nifti,v3.1.0,1.23.5,2.5.0,1.6.0,3.9.22,"{'minimumROIDimensions': 2, 'minimumROISize': ...",{'Original': {}},15b716c2bdac0f690e117aa8a956268648e74f78,3D,...,853.384800,0.003132,7.153493,0.557800,7.933386e+00,0.313949,0.002135,9100.546126,0.366337,3.794287
4,UCSF-PDGM-0149_nifti,v3.1.0,1.23.5,2.5.0,1.6.0,3.9.22,"{'minimumROIDimensions': 2, 'minimumROISize': ...",{'Original': {}},aeb6ea8e44b54092800c51b5c774131d335deb3f,3D,...,0.007016,0.007016,2.321928,0.008787,1.694736e+04,0.000000,1000000.000000,0.000000,0.000000,0.000000


In [4]:
# Load both CSVs
df_radiomics = pd.read_csv('radiomics_features_all_patients_TUMOR.csv')
df_metadata = pd.read_csv('UCSF-PDGM-metadata_v2.csv')

# Standardize column names
df_radiomics.rename(columns=lambda x: x.strip(), inplace=True)
df_metadata.rename(columns=lambda x: x.strip(), inplace=True)

# Strip and normalize IDs
df_radiomics['PatientID'] = df_radiomics['PatientID'].astype(str).str.strip().str.replace('_nifti', '', regex=False).str.upper()
df_metadata['PatientID'] = df_metadata['PatientID'].astype(str).str.strip().str.upper()

In [5]:
common_ids = set(df_radiomics['PatientID']) & set(df_metadata['PatientID'])

print("🧾 Radiomics PatientIDs:", df_radiomics['PatientID'].nunique())
print("🧾 Metadata PatientIDs:", df_metadata['PatientID'].nunique())
print("🔗 Common PatientIDs found after cleaning:", len(common_ids))

🧾 Radiomics PatientIDs: 217
🧾 Metadata PatientIDs: 501
🔗 Common PatientIDs found after cleaning: 217


In [6]:
df_merged = pd.merge(df_radiomics, df_metadata, on='PatientID', how='inner')
print("✅ Merged DataFrame shape:", df_merged.shape)

✅ Merged DataFrame shape: (217, 1693)


In [7]:
df_merged.to_csv("merged_radiomics_metadata.csv", index=False)
print("📁 Merged CSV saved as 'merged_radiomics_metadata.csv'")

📁 Merged CSV saved as 'merged_radiomics_metadata.csv'


---

# Model training

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier, VotingClassifier, StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.metrics import classification_report
from imblearn.over_sampling import SMOTE
import xgboost as xgb

# Step 1: Load merged data
df = pd.read_csv('merged_radiomics_metadata.csv')

# Step 2: Define features (X) and target (y)
X = df.drop(columns=["OS", "Survival_Category"], errors="ignore")
if "1-dead 0-alive" in df.columns:
    y = df["1-dead 0-alive"]
else:
    raise ValueError("Target column '1-dead 0-alive' not found in the dataset.")

# Step 3: Encode categorical features
categorical_cols = X.select_dtypes(include=["object"]).columns
label_encoders = {}
for col in categorical_cols:
    le = LabelEncoder()
    X[col] = le.fit_transform(X[col].astype(str))
    label_encoders[col] = le

# Step 4: Handle missing values
imputer = SimpleImputer(strategy="median")
X_imputed = imputer.fit_transform(X)

# Step 5: Standardize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_imputed)

# Step 6: Apply SMOTE
print("Class distribution before SMOTE:\n", y.value_counts())
smote = SMOTE(random_state=42)
X_smote, y_smote = smote.fit_resample(X_scaled, y)
print("Class distribution after SMOTE:\n", pd.Series(y_smote).value_counts())

# Step 7: Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X_smote, y_smote, test_size=0.3, stratify=y_smote, random_state=42
)

# Step 8: Define base models
rf = RandomForestClassifier(n_estimators=15, max_depth=8, random_state=42)
xgb_clf = xgb.XGBClassifier(
    n_estimators=10, max_depth=3, learning_rate=0.01,
    subsample=0.8, colsample_bytree=0.8, use_label_encoder=False,
    eval_metric='logloss', random_state=42
)
logreg = LogisticRegression(max_iter=500)
# lda = LinearDiscriminantAnalysis()

# Step 9: Define ensemble models
voting = VotingClassifier(estimators=[
    ("rf", rf), ("xgb", xgb_clf), ("logreg", logreg)
], voting="hard")

stacking = StackingClassifier(
    estimators=[("rf", rf), ("xgb", xgb_clf), ("logreg", logreg)],
    final_estimator=LogisticRegression(max_iter=100)
)

# Step 10: Train and evaluate
print("\n🔧 Training Voting Classifier...")
voting.fit(X_train, y_train)
y_pred_voting = voting.predict(X_test)
print("\nVoting Classifier Report:")
print(classification_report(y_test, y_pred_voting))

print("\n🔧 Training Stacking Classifier...")
stacking.fit(X_train, y_train)
y_pred_stacking = stacking.predict(X_test)
print("\nStacking Classifier Report:")
print(classification_report(y_test, y_pred_stacking))


Top 200 Features:
 ['IDH', 'Age at MRI', 'PatientID', 'MGMT index']
Class distribution before SMOTE:
 1-dead 0-alive
1    119
0     98
Name: count, dtype: int64
Class distribution after SMOTE:
 1-dead 0-alive
0    119
1    119
Name: count, dtype: int64

🔧 Training Voting Classifier...

Voting Classifier Report:
              precision    recall  f1-score   support

           0       0.74      0.69      0.71        36
           1       0.71      0.75      0.73        36

    accuracy                           0.72        72
   macro avg       0.72      0.72      0.72        72
weighted avg       0.72      0.72      0.72        72


🔧 Training Stacking Classifier...

Stacking Classifier Report:
              precision    recall  f1-score   support

           0       0.68      0.72      0.70        36
           1       0.71      0.67      0.69        36

    accuracy                           0.69        72
   macro avg       0.70      0.69      0.69        72
weighted avg       0.70

d:\mlpr data\.venv\lib\site-packages\xgboost\training.py:183: UserWarning: [18:41:10] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
d:\mlpr data\.venv\lib\site-packages\xgboost\training.py:183: UserWarning: [18:41:10] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
d:\mlpr data\.venv\lib\site-packages\xgboost\training.py:183: UserWarning: [18:41:10] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


In [13]:
correlations = df.corr(numeric_only=True)
print(correlations["1-dead 0-alive"].sort_values(ascending=False))

1-dead 0-alive                          1.000000
WHO CNS Grade                           0.404754
Age at MRI                              0.354230
L1_original_firstorder_Skewness         0.233038
SWI_original_gldm_DependenceVariance    0.203031
                                          ...   
L1_diagnostics_Image-original_Mean     -0.225585
L1_original_firstorder_10Percentile    -0.234841
ADC_original_firstorder_Kurtosis       -0.240387
MGMT index                             -0.248263
OS                                     -0.253029
Name: 1-dead 0-alive, Length: 1461, dtype: float64


In [31]:
rf = RandomForestClassifier(n_estimators=100, max_depth=15, random_state=42)
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)
print("\nRandom Forest Report:")
print(classification_report(y_test, y_pred_rf))


Random Forest Report:
              precision    recall  f1-score   support

           0       0.89      0.86      0.87        36
           1       0.86      0.89      0.88        36

    accuracy                           0.88        72
   macro avg       0.88      0.88      0.87        72
weighted avg       0.88      0.88      0.87        72



In [33]:
xgb_clf = xgb.XGBClassifier(
    n_estimators=10, max_depth=3, learning_rate=0.1,
    subsample=0.5, colsample_bytree=0.5, use_label_encoder=False,
    eval_metric='logloss', random_state=42
)
xgb_clf.fit(X_train, y_train)
y_pred_xgb = xgb_clf.predict(X_test)
print("\nXGBoost Report:")
print(classification_report(y_test, y_pred_xgb))


XGBoost Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        36
           1       1.00      1.00      1.00        36

    accuracy                           1.00        72
   macro avg       1.00      1.00      1.00        72
weighted avg       1.00      1.00      1.00        72



C:\Users\Devyansh\AppData\Roaming\Python\Python312\site-packages\xgboost\training.py:183: UserWarning: [23:11:03] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


In [34]:
logreg = LogisticRegression(max_iter=500)
logreg.fit(X_train, y_train)
y_pred_logreg = logreg.predict(X_test)
print("\nLogistic Regression Report:")
print(classification_report(y_test, y_pred_logreg))


Logistic Regression Report:
              precision    recall  f1-score   support

           0       0.89      0.94      0.92        36
           1       0.94      0.89      0.91        36

    accuracy                           0.92        72
   macro avg       0.92      0.92      0.92        72
weighted avg       0.92      0.92      0.92        72



In [ ]:
from sklearn.svm import SVC

svc = SVC(kernel='rbf', probability=True)
svc.fit(X_train, y_train)
y_pred_svc = svc.predict(X_test)
print("\nSupport Vector Classifier Report:")
print(classification_report(y_test, y_pred_svc))


Support Vector Classifier Report:
              precision    recall  f1-score   support

           0       0.71      0.56      0.62        36
           1       0.64      0.78      0.70        36

    accuracy                           0.67        72
   macro avg       0.68      0.67      0.66        72
weighted avg       0.68      0.67      0.66        72



In [42]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier, VotingClassifier, StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from imblearn.over_sampling import SMOTE

# Step 1: Load merged data
df = pd.read_csv('merged_radiomics_metadata.csv')

# Step 2: Define features (X) and target (y)
X = df.drop(columns=["OS", "Survival_Category"], errors="ignore")
if "1-dead 0-alive" in df.columns:
    y = df["1-dead 0-alive"]
else:
    raise ValueError("Target column '1-dead 0-alive' not found in the dataset.")

# Step 3: Encode categorical features
categorical_cols = X.select_dtypes(include=["object"]).columns
label_encoders = {}
for col in categorical_cols:
    le = LabelEncoder()
    X[col] = le.fit_transform(X[col].astype(str))
    label_encoders[col] = le

# Step 4: Handle missing values
imputer = SimpleImputer(strategy="median")
X_imputed = imputer.fit_transform(X)

# Step 5: Standardize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_imputed)

# Step 6: Apply SMOTE
print("Class distribution before SMOTE:\n", y.value_counts())
smote = SMOTE(random_state=42)
X_smote, y_smote = smote.fit_resample(X_scaled, y)
print("Class distribution after SMOTE:\n", pd.Series(y_smote).value_counts())

# Step 7: Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X_smote, y_smote, test_size=0.3, stratify=y_smote, random_state=42
)

# Step 8: Define base models
rf = RandomForestClassifier(n_estimators=50, max_depth=10, random_state=42)
svc = SVC(kernel='poly', C=1.0, probability=True, random_state=42)
logreg = LogisticRegression(max_iter=100)
lda = LinearDiscriminantAnalysis()

# Step 9: Define ensemble models
voting = VotingClassifier(estimators=[
    ("rf", rf), ("svc", svc), ("logreg", logreg), ("lda", lda)
], voting="hard")

stacking = StackingClassifier(
    estimators=[("rf", rf), ("svc", svc), ("logreg", logreg), ("lda", lda)],
    final_estimator=LogisticRegression(max_iter=100)
)

# Step 10: Train and evaluate
print("\n🔧 Training Voting Classifier...")
voting.fit(X_train, y_train)
y_pred_voting = voting.predict(X_test)
print("\nVoting Classifier Report:")
print(classification_report(y_test, y_pred_voting))

print("\n🔧 Training Stacking Classifier...")
stacking.fit(X_train, y_train)
y_pred_stacking = stacking.predict(X_test)
print("\nStacking Classifier Report:")
print(classification_report(y_test, y_pred_stacking))

Class distribution before SMOTE:
 1-dead 0-alive
1    119
0     98
Name: count, dtype: int64
Class distribution after SMOTE:
 1-dead 0-alive
0    119
1    119
Name: count, dtype: int64

🔧 Training Voting Classifier...

Voting Classifier Report:
              precision    recall  f1-score   support

           0       0.90      0.97      0.93        36
           1       0.97      0.89      0.93        36

    accuracy                           0.93        72
   macro avg       0.93      0.93      0.93        72
weighted avg       0.93      0.93      0.93        72


🔧 Training Stacking Classifier...

Stacking Classifier Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        36
           1       1.00      1.00      1.00        36

    accuracy                           1.00        72
   macro avg       1.00      1.00      1.00        72
weighted avg       1.00      1.00      1.00        72



---

# Using LDA to reduce features

In [7]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier, VotingClassifier, StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.svm import SVC
from sklearn.metrics import (
    accuracy_score, balanced_accuracy_score, f1_score,
    roc_auc_score, precision_score, recall_score
)
from sklearn.feature_selection import SelectKBest, f_classif
from imblearn.over_sampling import SMOTE
from lifelines.utils import concordance_index

# Step 1: Load merged data
df = pd.read_csv('merged_radiomics_metadata.csv')

# Step 2: Define features (X) and target (y)
X = df.drop(columns=["OS", "Survival_Category"], errors="ignore")
if "1-dead 0-alive" in df.columns:
    y = df["1-dead 0-alive"]
else:
    raise ValueError("Target column '1-dead 0-alive' not found in the dataset.")

# Optional: For Concordance Index
survival_times = df["OS"].values if "OS" in df.columns else None

# Step 3: Encode categorical features
categorical_cols = X.select_dtypes(include=["object"]).columns
label_encoders = {}
for col in categorical_cols:
    le = LabelEncoder()
    X[col] = le.fit_transform(X[col].astype(str))
    label_encoders[col] = le

# Step 4: Handle missing values
imputer = SimpleImputer(strategy="median")
X_imputed = imputer.fit_transform(X)

# Step 5: Standardize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_imputed)

# Step 6: Apply SMOTE
print("Class distribution before SMOTE:\n", y.value_counts())
smote = SMOTE(random_state=42)
X_smote, y_smote = smote.fit_resample(X_scaled, y)
print("Class distribution after SMOTE:\n", pd.Series(y_smote).value_counts())

# Step 7: Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X_smote, y_smote, test_size=0.3, stratify=y_smote, random_state=42
)

# Step 8: Define base models
rf = RandomForestClassifier(n_estimators=50, max_depth=10, random_state=42)
svc = SVC(kernel='poly', C=1.0, probability=True, random_state=42)
logreg = LogisticRegression(max_iter=100)
lda = LinearDiscriminantAnalysis()

# Step 9: Define ensemble models
voting = VotingClassifier(estimators=[
    ("rf", rf), ("svc", svc), ("logreg", logreg), ("lda", lda)
], voting="soft")

stacking = StackingClassifier(
    estimators=[("rf", rf), ("svc", svc), ("logreg", logreg), ("lda", lda)],
    final_estimator=LogisticRegression(max_iter=100)
)

# Step 10: Evaluation function
def evaluate_model(y_true, y_pred, y_proba=None, survival_time=None):
    print("Accuracy:", accuracy_score(y_true, y_pred))
    print("Balanced Accuracy:", balanced_accuracy_score(y_true, y_pred))
    print("F1 Score:", f1_score(y_true, y_pred))
    print("Precision:", precision_score(y_true, y_pred))
    print("Recall:", recall_score(y_true, y_pred))
    if y_proba is not None:
        print("ROC-AUC:", roc_auc_score(y_true, y_proba))
    if survival_time is not None:
        try:
            ci = concordance_index(survival_time, -y_proba)
            print("Concordance Index:", ci)
        except:
            print("Concordance Index: NA (Error computing with provided survival time)")

# Step 11: Feature reduction and evaluation loop
for k in [1693, 1600, 1500, 1400]:
    print(f"\n🔎 Selecting Top {k} Features using ANOVA F-test...")
    selector = SelectKBest(score_func=f_classif, k=k)
    X_train_k = selector.fit_transform(X_train, y_train)
    X_test_k = selector.transform(X_test)

    # Voting Classifier
    print(f"\n📦 Training Voting Classifier with {k} features...")
    voting.fit(X_train_k, y_train)
    y_pred_voting = voting.predict(X_test_k)
    y_proba_voting = voting.predict_proba(X_test_k)[:, 1]

    print("📊 Voting Classifier Metrics:")
    evaluate_model(y_test, y_pred_voting, y_proba_voting, survival_time=survival_times)

    # Stacking Classifier
    print(f"\n📦 Training Stacking Classifier with {k} features...")
    stacking.fit(X_train_k, y_train)
    y_pred_stacking = stacking.predict(X_test_k)
    y_proba_stacking = stacking.predict_proba(X_test_k)[:, 1]

    print("📊 Stacking Classifier Metrics:")
    evaluate_model(y_test, y_pred_stacking, y_proba_stacking, survival_time=survival_times)

Class distribution before SMOTE:
 1-dead 0-alive
1    119
0     98
Name: count, dtype: int64
Class distribution after SMOTE:
 1-dead 0-alive
0    119
1    119
Name: count, dtype: int64

🔎 Selecting Top 1693 Features using ANOVA F-test...

📦 Training Voting Classifier with 1693 features...


C:\Users\Devyansh\AppData\Roaming\Python\Python312\site-packages\sklearn\feature_selection\_univariate_selection.py:776: UserWarning: k=1693 is greater than n_features=1692. All the features will be returned.
  warnings.warn(


📊 Voting Classifier Metrics:
Accuracy: 0.9861111111111112
Balanced Accuracy: 0.9861111111111112
F1 Score: 0.9859154929577465
Precision: 1.0
Recall: 0.9722222222222222
ROC-AUC: 1.0
Concordance Index: NA (Error computing with provided survival time)

📦 Training Stacking Classifier with 1693 features...
📊 Stacking Classifier Metrics:
Accuracy: 1.0
Balanced Accuracy: 1.0
F1 Score: 1.0
Precision: 1.0
Recall: 1.0
ROC-AUC: 1.0
Concordance Index: NA (Error computing with provided survival time)

🔎 Selecting Top 1600 Features using ANOVA F-test...

📦 Training Voting Classifier with 1600 features...
📊 Voting Classifier Metrics:
Accuracy: 0.6388888888888888
Balanced Accuracy: 0.6388888888888888
F1 Score: 0.6388888888888888
Precision: 0.6388888888888888
Recall: 0.6388888888888888
ROC-AUC: 0.7337962962962963
Concordance Index: NA (Error computing with provided survival time)

📦 Training Stacking Classifier with 1600 features...


C:\Users\Devyansh\AppData\Roaming\Python\Python312\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


📊 Stacking Classifier Metrics:
Accuracy: 0.6944444444444444
Balanced Accuracy: 0.6944444444444444
F1 Score: 0.6944444444444444
Precision: 0.6944444444444444
Recall: 0.6944444444444444
ROC-AUC: 0.7631172839506173
Concordance Index: NA (Error computing with provided survival time)

🔎 Selecting Top 1500 Features using ANOVA F-test...

📦 Training Voting Classifier with 1500 features...
📊 Voting Classifier Metrics:
Accuracy: 0.7222222222222222
Balanced Accuracy: 0.7222222222222222
F1 Score: 0.7368421052631579
Precision: 0.7
Recall: 0.7777777777777778
ROC-AUC: 0.7515432098765432
Concordance Index: NA (Error computing with provided survival time)

📦 Training Stacking Classifier with 1500 features...
📊 Stacking Classifier Metrics:
Accuracy: 0.6944444444444444
Balanced Accuracy: 0.6944444444444444
F1 Score: 0.6944444444444444
Precision: 0.6944444444444444
Recall: 0.6944444444444444
ROC-AUC: 0.7746913580246914
Concordance Index: NA (Error computing with provided survival time)

🔎 Selecting Top 1

In [9]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier, VotingClassifier, StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.svm import SVC
from sklearn.metrics import (
    accuracy_score, balanced_accuracy_score, f1_score,
    roc_auc_score, precision_score, recall_score
)
from sklearn.feature_selection import SelectKBest, f_classif
from imblearn.over_sampling import SMOTE
from lifelines.utils import concordance_index

# Load data
df = pd.read_csv('merged_radiomics_metadata.csv')
X = df.drop(columns=["OS", "Survival_Category"], errors="ignore")
if "1-dead 0-alive" in df.columns:
    y = df["1-dead 0-alive"]
else:
    raise ValueError("Target column '1-dead 0-alive' not found.")
survival_times = df["OS"].values if "OS" in df.columns else None

# Encode categoricals
categorical_cols = X.select_dtypes(include=["object"]).columns
for col in categorical_cols:
    X[col] = LabelEncoder().fit_transform(X[col].astype(str))

# Impute & scale
X_imputed = SimpleImputer(strategy="median").fit_transform(X)
X_scaled = StandardScaler().fit_transform(X_imputed)

# SMOTE
print("Before SMOTE:\n", y.value_counts())
X_smote, y_smote = SMOTE(random_state=42).fit_resample(X_scaled, y)
print("After SMOTE:\n", pd.Series(y_smote).value_counts())

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X_smote, y_smote, test_size=0.3, stratify=y_smote, random_state=42
)

# Base models
rf = RandomForestClassifier(n_estimators=50, max_depth=10, random_state=42)
svc = SVC(kernel='poly', C=1.0, probability=True, random_state=42)
logreg = LogisticRegression(max_iter=100)
lda = LinearDiscriminantAnalysis()

# Ensembles
voting = VotingClassifier(estimators=[
    ("rf", rf), ("svc", svc), ("logreg", logreg), ("lda", lda)
], voting="soft")

stacking = StackingClassifier(
    estimators=[("rf", rf), ("svc", svc), ("logreg", logreg), ("lda", lda)],
    final_estimator=LogisticRegression(max_iter=100)
)

# Evaluation function
def evaluate_model(y_true, y_pred, y_proba=None, survival_time=None):
    metrics = {}
    metrics["Accuracy"] = accuracy_score(y_true, y_pred)
    metrics["Balanced Accuracy"] = balanced_accuracy_score(y_true, y_pred)
    metrics["F1"] = f1_score(y_true, y_pred)
    metrics["Precision"] = precision_score(y_true, y_pred)
    metrics["Recall"] = recall_score(y_true, y_pred)
    if y_proba is not None:
        metrics["ROC-AUC"] = roc_auc_score(y_true, y_proba)
    if survival_time is not None and y_proba is not None:
        try:
            metrics["C-Index"] = concordance_index(survival_time, -y_proba)
        except:
            metrics["C-Index"] = None
    return metrics

# Feature search loop
best_k = None
best_auc = 0
best_results = {}
k_values = [1693, 1650, 250, 200, 900, 500]

for k in k_values:
    print(f"\n🔎 Evaluating Top {k} Features...")
    selector = SelectKBest(score_func=f_classif, k=k)
    X_train_k = selector.fit_transform(X_train, y_train)
    X_test_k = selector.transform(X_test)

    voting.fit(X_train_k, y_train)
    y_pred = voting.predict(X_test_k)
    y_proba = voting.predict_proba(X_test_k)[:, 1]

    results = evaluate_model(y_test, y_pred, y_proba, survival_times)
    print("Metrics:", results)

    if results.get("ROC-AUC", 0) > best_auc:
        best_auc = results["ROC-AUC"]
        best_k = k
        best_results = results
        best_selector = selector  # Store for final use

# Final evaluation on best features
print(f"\n🏆 Best feature count: {best_k} with ROC-AUC: {best_auc:.4f}")
X_train_best = best_selector.transform(X_train)
X_test_best = best_selector.transform(X_test)

# Re-train both ensembles
voting.fit(X_train_best, y_train)
stacking.fit(X_train_best, y_train)

# Voting Evaluation
y_pred_v = voting.predict(X_test_best)
y_proba_v = voting.predict_proba(X_test_best)[:, 1]
print("\n📊 Final Voting Classifier Metrics:")
final_voting_results = evaluate_model(y_test, y_pred_v, y_proba_v, survival_times)
print(final_voting_results)

# Stacking Evaluation
y_pred_s = stacking.predict(X_test_best)
y_proba_s = stacking.predict_proba(X_test_best)[:, 1]
print("\n📊 Final Stacking Classifier Metrics:")
final_stacking_results = evaluate_model(y_test, y_pred_s, y_proba_s, survival_times)
print(final_stacking_results)


Before SMOTE:
 1-dead 0-alive
1    119
0     98
Name: count, dtype: int64
After SMOTE:
 1-dead 0-alive
0    119
1    119
Name: count, dtype: int64

🔎 Evaluating Top 1693 Features...


C:\Users\Devyansh\AppData\Roaming\Python\Python312\site-packages\sklearn\feature_selection\_univariate_selection.py:776: UserWarning: k=1693 is greater than n_features=1692. All the features will be returned.
  warnings.warn(


Metrics: {'Accuracy': 0.9861111111111112, 'Balanced Accuracy': 0.9861111111111112, 'F1': 0.9859154929577465, 'Precision': 1.0, 'Recall': 0.9722222222222222, 'ROC-AUC': 1.0, 'C-Index': None}

🔎 Evaluating Top 1650 Features...
Metrics: {'Accuracy': 0.625, 'Balanced Accuracy': 0.625, 'F1': 0.6086956521739131, 'Precision': 0.6363636363636364, 'Recall': 0.5833333333333334, 'ROC-AUC': 0.7546296296296297, 'C-Index': None}

🔎 Evaluating Top 250 Features...
Metrics: {'Accuracy': 0.6805555555555556, 'Balanced Accuracy': 0.6805555555555556, 'F1': 0.676056338028169, 'Precision': 0.6857142857142857, 'Recall': 0.6666666666666666, 'ROC-AUC': 0.7299382716049383, 'C-Index': None}

🔎 Evaluating Top 200 Features...
Metrics: {'Accuracy': 0.6666666666666666, 'Balanced Accuracy': 0.6666666666666666, 'F1': 0.6756756756756757, 'Precision': 0.6578947368421053, 'Recall': 0.6944444444444444, 'ROC-AUC': 0.7592592592592593, 'C-Index': None}

🔎 Evaluating Top 900 Features...
Metrics: {'Accuracy': 0.6944444444444444